In [1]:
import requests
import json
import datetime
import os

In [2]:
key = ''
with open('solar_edge_api_key.txt') as file:
    key = file.read().replace('\n', '')
key

'Q5NGC734DJB0XPIR6L6K9OZPWQ5YOE9U'

In [4]:
site_info = ''
with open('site_information.json') as file:
    site_info = file.read()
site_info = json.loads(site_info)

In [5]:
base_url = 'https://monitoringapi.solaredge.com/'

In [6]:
def last_day_of_month(any_day):
    next_month = any_day.replace(day=28) + datetime.timedelta(days=4)  # this will never fail
    return next_month - datetime.timedelta(days=next_month.day)

In [7]:
class Site():
    def __init__(self, id, name, accountId, data_period_url):
        self.id = id
        self.name = name
        self.accountId = accountId
        self.data_period_url = data_period_url
        self.start = ''
        self.end = ''
        
    def get_start_end_dates(self):
        url = f'{base_url}{self.data_period_url}?api_key={key}'
        response = requests.get(url)
        r_json = response.json()
        self.start = r_json['dataPeriod']['startDate']
        self.end = r_json['dataPeriod']['endDate']
        
    def get_month_periods(self):
        periods = []
        first = datetime.datetime.strptime(self.start, "%Y-%m-%d")
        last = last_day_of_month(first)
        periods.append((first,last))
        end = datetime.datetime.strptime(self.end, "%Y-%m-%d")
        loop = True
        while loop:
            first = last + datetime.timedelta(days=1)
            last = last_day_of_month(first)
            if last > end:
                last = end
                loop = False
            periods.append((first,last))
        self.periods = periods

Make sure not to call this every time to conserve api calls

In [8]:
# sites = []
# for site in site_info['sites']['site']:
#     new_site = Site(site['id'], site['name'], site['accountId'], site['uris']['DATA_PERIOD'])
#     new_site.get_start_end_dates()
#     sites.append(new_site)
    

Saving for future use so I don't have to ping the API every time, trying to conserve on calls. 

In [9]:
# csv_string = 'id,name,accountId,DATA_PERIOD,startDate,endDate\n'
# for site in sites:
#     csv_string = csv_string + f'{site.id},{site.name},{site.accountId},{site.data_period_url},{site.start},{site.end}\n'

# with open('site_information.csv', 'w') as file:
#     file.write(csv_string)

In [10]:
import csv
sites = []
with open('site_information.csv', mode='r') as infile:
    reader = csv.DictReader(infile)
    for row in reader:
        site = Site(row['id'], row['name'], row['accountId'], row['DATA_PERIOD'])
        site.start = row['startDate']
        print(site.start)
        site.end = row['endDate']
        site.get_month_periods()
        sites.append(site)
# print(sites)

2015-02-09
2019-08-14
2018-07-23
2016-03-31
2016-12-28
2017-03-09
2018-02-09
2018-11-06
2018-04-21
2018-01-17
2017-10-23
2017-02-24
2016-05-16
2017-06-08


We want to get the month ranges for each month so that we can get the 15 minute time stamp production data for each site.

In [11]:
for site in sites:
    print(site.name)

print(sites[2].name)
print(sites[2].periods)

Alan Knudson House
Cherry Orchards Lot#33
Crop Circles Carport
Dave Meyers
Francisco Garcia 1
Gabriel Pineiro
Garkane Kanab Pavilion
Huntington Beach
Lion House
Michael Cawley
Quail Creek Ranch
Ronald L. McClellan Sr.
Sam Zitting
SunGrow Wilkins and Juniper
Crop Circles Carport
[(datetime.datetime(2018, 7, 23, 0, 0), datetime.datetime(2018, 7, 31, 0, 0)), (datetime.datetime(2018, 8, 1, 0, 0), datetime.datetime(2018, 8, 31, 0, 0)), (datetime.datetime(2018, 9, 1, 0, 0), datetime.datetime(2018, 9, 30, 0, 0)), (datetime.datetime(2018, 10, 1, 0, 0), datetime.datetime(2018, 10, 31, 0, 0)), (datetime.datetime(2018, 11, 1, 0, 0), datetime.datetime(2018, 11, 30, 0, 0)), (datetime.datetime(2018, 12, 1, 0, 0), datetime.datetime(2018, 12, 31, 0, 0)), (datetime.datetime(2019, 1, 1, 0, 0), datetime.datetime(2019, 1, 31, 0, 0)), (datetime.datetime(2019, 2, 1, 0, 0), datetime.datetime(2019, 2, 28, 0, 0)), (datetime.datetime(2019, 3, 1, 0, 0), datetime.datetime(2019, 3, 31, 0, 0)), (datetime.datetime(2

test one of the date ranges to see if the api wants to complain

In [13]:
def make_energy_call_for_range(id, start, end):
    start = start.strftime('%Y-%m-%d')
    end = end.strftime('%Y-%m-%d')
    url = f'{base_url}/site/{id}/energy?timeUnit=QUARTER_OF_AN_HOUR&startDate={start}&endDate={end}&api_key={key}'
    response = requests.get(url)
    print(response)
    r_json = response.json()
    return r_json

In [14]:
site = sites[0]
start = site.periods[1][0]
end = site.periods[1][1]
mydict = make_energy_call_for_range(site.id, start, end)
print(mydict)

<Response [200]>
{'energy': {'timeUnit': 'QUARTER_OF_AN_HOUR', 'unit': 'Wh', 'measuredBy': 'INVERTER', 'values': [{'date': '2015-03-01 00:00:00', 'value': None}, {'date': '2015-03-01 00:15:00', 'value': None}, {'date': '2015-03-01 00:30:00', 'value': None}, {'date': '2015-03-01 00:45:00', 'value': None}, {'date': '2015-03-01 01:00:00', 'value': None}, {'date': '2015-03-01 01:15:00', 'value': None}, {'date': '2015-03-01 01:30:00', 'value': None}, {'date': '2015-03-01 01:45:00', 'value': None}, {'date': '2015-03-01 02:00:00', 'value': None}, {'date': '2015-03-01 02:15:00', 'value': None}, {'date': '2015-03-01 02:30:00', 'value': None}, {'date': '2015-03-01 02:45:00', 'value': None}, {'date': '2015-03-01 03:00:00', 'value': None}, {'date': '2015-03-01 03:15:00', 'value': None}, {'date': '2015-03-01 03:30:00', 'value': None}, {'date': '2015-03-01 03:45:00', 'value': None}, {'date': '2015-03-01 04:00:00', 'value': None}, {'date': '2015-03-01 04:15:00', 'value': None}, {'date': '2015-03-01 0

Loop through each period for each site, ping the api, and write the data to a csv file

In [15]:
def write_to_csv(site, start, end, data):
    filepath = f'./data/{site.name}/'
    start = start.strftime("%Y-%m-%d")
    end = end.strftime("%Y-%m-%d")
    name = f'{start}_to_{end}_15_min_resolution.csv'
    print(name)
    if not os.path.exists(filepath):
        os.makedirs(filepath)
    
    filename = f'{filepath}{name}'
    with open(filename, 'w', newline='') as csvfile:
        fieldnames = data[0].keys()
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()
        writer.writerows(data)

In [16]:
write_to_csv(site, start, end, mydict['energy']['values'])

2015-03-01_to_2015-03-31_15_min_resolution.csv


In [31]:
for period in site.periods:
    start = period[0]
    end = period[1]
    mydict = make_energy_call_for_range(site.id, start, end)
    write_to_csv(site, start, end, mydict['energy']['values'])

<Response [200]>
2015-02-09_to_2015-02-28_15_min_resolution.csv
<Response [200]>
2015-03-01_to_2015-03-31_15_min_resolution.csv
<Response [200]>
2015-04-01_to_2015-04-30_15_min_resolution.csv
<Response [200]>
2015-05-01_to_2015-05-31_15_min_resolution.csv
<Response [200]>
2015-06-01_to_2015-06-30_15_min_resolution.csv
<Response [200]>
2015-07-01_to_2015-07-31_15_min_resolution.csv
<Response [200]>
2015-08-01_to_2015-08-31_15_min_resolution.csv
<Response [200]>
2015-09-01_to_2015-09-30_15_min_resolution.csv
<Response [200]>
2015-10-01_to_2015-10-31_15_min_resolution.csv
<Response [200]>
2015-11-01_to_2015-11-30_15_min_resolution.csv
<Response [200]>
2015-12-01_to_2015-12-31_15_min_resolution.csv
<Response [200]>
2016-01-01_to_2016-01-31_15_min_resolution.csv
<Response [200]>
2016-02-01_to_2016-02-29_15_min_resolution.csv
<Response [200]>
2016-03-01_to_2016-03-31_15_min_resolution.csv
<Response [200]>
2016-04-01_to_2016-04-30_15_min_resolution.csv
<Response [200]>
2016-05-01_to_2016-05-3

In [17]:
# Lion House
# Michael Cawley
# Quail Creek Ranch
# Ronald L. McClellan Sr.
# Sam Zitting
# SunGrow Wilkins and Juniper
# Crop Circles Carport

site_names = [
    'Lion House', 
    'Michael Cawley', 
    'Quail Creek Ranch', 
    'Ronald L. McClellan Sr.', 
    'Sam Zitting', 
    'SunGrow Wilkins and Juniper',
    'Crop Circles Carport'
]

for site in sites:
#     if site.name != 'Alan Knudson House':
    if site.name in site_names:
        for period in site.periods:
            start = period[0]
            end = period[1]
            mydict = make_energy_call_for_range(site.id, start, end)
            write_to_csv(site, start, end, mydict['energy']['values'])

<Response [200]>
2018-07-23_to_2018-07-31_15_min_resolution.csv
<Response [200]>
2018-08-01_to_2018-08-31_15_min_resolution.csv
<Response [200]>
2018-09-01_to_2018-09-30_15_min_resolution.csv
<Response [200]>
2018-10-01_to_2018-10-31_15_min_resolution.csv
<Response [200]>
2018-11-01_to_2018-11-30_15_min_resolution.csv
<Response [200]>
2018-12-01_to_2018-12-31_15_min_resolution.csv
<Response [200]>
2019-01-01_to_2019-01-31_15_min_resolution.csv
<Response [200]>
2019-02-01_to_2019-02-28_15_min_resolution.csv
<Response [200]>
2019-03-01_to_2019-03-31_15_min_resolution.csv
<Response [200]>
2019-04-01_to_2019-04-30_15_min_resolution.csv
<Response [200]>
2019-05-01_to_2019-05-31_15_min_resolution.csv
<Response [200]>
2019-06-01_to_2019-06-30_15_min_resolution.csv
<Response [200]>
2019-07-01_to_2019-07-31_15_min_resolution.csv
<Response [200]>
2019-08-01_to_2019-08-31_15_min_resolution.csv
<Response [200]>
2019-09-01_to_2019-09-30_15_min_resolution.csv
<Response [200]>
2019-10-01_to_2019-10-3

<Response [200]>
2020-01-01_to_2020-01-21_15_min_resolution.csv
<Response [200]>
2016-05-16_to_2016-05-31_15_min_resolution.csv
<Response [200]>
2016-06-01_to_2016-06-30_15_min_resolution.csv
<Response [200]>
2016-07-01_to_2016-07-31_15_min_resolution.csv
<Response [200]>
2016-08-01_to_2016-08-31_15_min_resolution.csv
<Response [200]>
2016-09-01_to_2016-09-30_15_min_resolution.csv
<Response [200]>
2016-10-01_to_2016-10-31_15_min_resolution.csv
<Response [200]>
2016-11-01_to_2016-11-30_15_min_resolution.csv
<Response [200]>
2016-12-01_to_2016-12-31_15_min_resolution.csv
<Response [200]>
2017-01-01_to_2017-01-31_15_min_resolution.csv
<Response [200]>
2017-02-01_to_2017-02-28_15_min_resolution.csv
<Response [200]>
2017-03-01_to_2017-03-31_15_min_resolution.csv
<Response [200]>
2017-04-01_to_2017-04-30_15_min_resolution.csv
<Response [200]>
2017-05-01_to_2017-05-31_15_min_resolution.csv
<Response [200]>
2017-06-01_to_2017-06-30_15_min_resolution.csv
<Response [200]>
2017-07-01_to_2017-07-3